<a href="https://colab.research.google.com/github/virtualthree/Plant-Disease-Detection/blob/main/Feature-Extraction-Maize(MobileNet-DenseNet).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from collections import Counter
import cv2
import time
import glob
import skimage
import numpy as np
import pandas as pd
import seaborn as sn
from tqdm import tqdm
from os import listdir
import matplotlib.pyplot as plt
from skimage.transform import resize
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

# !pip install FS
from sklearn.neighbors import KNeighborsClassifier
from drive.MyDrive.FS.woa import jfs   # change this to switch algorithm 

sn.set()

from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from time import time

from sklearn import metrics
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score,f1_score,recall_score,cohen_kappa_score,precision_score
from sklearn.utils import compute_class_weight
from sklearn.preprocessing import MinMaxScaler,LabelBinarizer, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier # KNeighborsClassifier
from sklearn.svm import SVC # SVM
from sklearn.ensemble import RandomForestClassifier # RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier # AdaBoostClassifier
from xgboost import XGBClassifier # XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import tensorflow 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import AUC
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.vgg16 import VGG16 # VGG16
from tensorflow.keras.applications.vgg19 import VGG19 # VGG19
from tensorflow.keras.applications.resnet50 import ResNet50 # ResNet50
#from tensorflow.keras.applications.resnet9 import ResNet9 # ResNet50
from tensorflow.keras.applications import ResNet101,InceptionResNetV2 # ResNet 101
from tensorflow.keras.applications.xception import Xception # Xception
from tensorflow.keras.applications.mobilenet import MobileNet # MobileNet
from tensorflow.keras.applications.densenet import DenseNet169 # DenseNet169
from tensorflow.keras.applications.densenet import DenseNet121 # DenseNet121
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 # MobileNetV2
from tensorflow.keras.applications.inception_v3 import InceptionV3 # InceptionV3
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Flatten, Activation, GlobalAveragePooling2D,Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
def loadImages(path):
    
    sample = []
    
    for filename in glob.glob(path):
        
        img = cv2.imread(filename)
        img = skimage.transform.resize(img, (224, 224, 3))
        IMG = np.array(img)
        sample.append(IMG)
        
    return sample

In [ ]:
train_path1 = 'drive/My Drive/Maize Dataset/train/Blight/*.jpg' 
train_path2 = 'drive/My Drive/Maize Dataset/train/Common_Rust/*.jpg'
train_path3 = 'drive/My Drive/Maize Dataset/train/Gray_Leaf_Spot/*.jpg'
train_path4 = 'drive/My Drive/Maize Dataset/train/Healthy/*.jpg'
train_BL = loadImages(train_path1)         
train_CR = loadImages(train_path2)                  
train_GLS = loadImages(train_path3)                 
train_HL = loadImages(train_path4)             

In [ ]:
df_train_ND = pd.DataFrame({'image':train_BL, 'label': 'BL'})           
df_train_VMD = pd.DataFrame({'image':train_CR, 'label': 'CR'})      
df_train_MID = pd.DataFrame({'image':train_GLS, 'label': 'GLS'})  
df_train_MOD = pd.DataFrame({'image':train_HL, 'label': 'HL'})  

final_data = [df_train_ND, df_train_VMD, df_train_MID, df_train_MOD] 
final_data = pd.concat(final_data)

print("Entire data size:",final_data.shape)

Entire data size: (1829, 2)


In [ ]:
#%% TRAIN LABEL SEPARATION

train_data = final_data['image']
labels = final_data['label']

print("Labels Count:",Counter(np.array(labels)))

onehot = LabelEncoder()
labels = onehot.fit_transform(labels)
print(labels.size)

X_train, X_test, y_train, y_test = train_test_split(train_data, labels,
                                                  test_size = 0.3,
                                                  stratify = labels,
                                                  )

X_train = np.array(X_train)
X_test = np.array(X_test)

print(y_train.size)

Labels Count: Counter({'3': 209, '7': 208, '10': 208, '8': 205, '9': 205, '4': 204, '2': 203, '6': 201, '5': 186})
1829
1280


In [ ]:
print(train_data.shape)
print(np.unique(y_train))

(1829,)
[0 1 2 3 4 5 6 7 8]


In [ ]:
x_train = np.empty((len(X_train),X_train[0].shape[0],X_train[0].shape[1],X_train[0].shape[2]))
for i,x in enumerate(X_train):
    x_train[i]=X_train[i]
print(x_train.shape)
x_test= np.empty((len(X_test),X_test[0].shape[0],X_test[0].shape[1],X_test[0].shape[2]))
for i,x in enumerate(X_test):
    x_test[i]=X_test[i]
print(x_test.shape)

(1280, 224, 224, 3)
(549, 224, 224, 3)


In [ ]:
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(224,224,3))
x = base_model.output
x =  Dropout(0.5)(x)
x = Flatten()(x)

model_feat = Model(inputs=base_model.input,outputs=x)

mobile_train_features = model_feat.predict(x_train)
mobile_test_features=model_feat.predict(x_test)
print(mobile_train_features)
print(mobile_test_features)

18/18 [==============================] - 349s 19s/step
[[0.07329067 0.         0.         ... 0.         0.5413985  0.        ]
 [0.13126302 0.         0.         ... 0.         0.5954092  0.        ]
 [0.16999844 0.         0.         ... 0.         0.71328527 0.        ]
 ...
 [0.35736436 0.         0.         ... 0.         0.6402618  0.        ]
 [0.06424177 0.         0.         ... 0.         0.35160396 0.        ]
 [0.24905534 0.         0.         ... 0.         0.6955431  0.        ]]
[[0.12966558 0.         0.         ... 0.         0.5389942  0.        ]
 [0.46841046 0.         0.         ... 0.         0.80501163 0.        ]
 [0.05661519 0.         0.         ... 0.         0.69082    0.        ]
 ...
 [0.20721866 0.         0.         ... 0.         0.6338099  0.        ]
 [0.27339017 0.         0.         ... 0.         0.5134727  0.        ]
 [0.11178973 0.         0.         ... 0.         0.6652419  0.        ]]


In [ ]:
base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(224,224,3))
y = base_model.output
y =  Dropout(0.5)(y)
y = Flatten()(y)

model_feat = Model(inputs=base_model.input,outputs=y)

dense_train_features = model_feat.predict(x_train)
dense_test_features=model_feat.predict(x_test)
print(dense_train_features)
print(dense_test_features)

18/18 [==============================] - 111s 6s/step
[[0.        0.        0.        ... 4.4602184 0.        0.       ]
 [0.        0.        0.        ... 4.199543  0.        0.       ]
 [0.        0.        0.        ... 4.755846  0.        0.       ]
 ...
 [0.        0.        0.        ... 4.586478  0.        0.       ]
 [0.        0.        0.        ... 5.20969   0.        0.       ]
 [0.        0.        0.        ... 4.556794  0.        0.       ]]
[[0.        0.        0.        ... 4.219782  0.        0.       ]
 [0.        0.        0.        ... 4.4952683 0.        0.       ]
 [0.        0.        0.        ... 4.3112087 0.        0.       ]
 ...
 [0.        0.        0.        ... 5.2517614 0.        0.       ]
 [0.        0.        0.        ... 4.4972963 0.        0.       ]
 [0.        0.        0.        ... 4.4127417 0.        0.       ]]


In [ ]:
csv_mobile = pd.DataFrame(mobile_train_features)  
csv_densenet = pd.DataFrame(dense_train_features)  
print(np.unique(y_train))
csv_label = pd.DataFrame(y_train) 
final_csv = [csv_mobile,csv_densenet,csv_label]  
#final_csv = pd.concat(final_csv,axis=1) 
#final_csv.to_csv('train_features_2.csv')  
new = pd.concat(final_csv, axis=1, join='inner')
new.to_csv('train_features_2.csv') 
!cp train_features_2.csv "drive/My Drive/"

[0 1 2 3 4 5 6 7 8]


In [ ]:
train_features = np.append(mobile_train_features,dense_train_features,axis=1)
test_features = np.append(mobile_test_features,dense_test_features,axis=1)

In [ ]:
x_train_acc, x_val_acc, y_train_acc, y_val_acc = train_test_split(train_features,y_train,
                                                  test_size = 0.2,
                                                  stratify = y_train,
                                                  shuffle = True,
                                                  random_state = 42)
X_test,y_test=test_features,y_test

print('length X_train:', len(x_train_acc))
print('length y_train:', len(y_train_acc))

print('length X_val:',  len(x_val_acc))
print('length y_val:', len(y_val_acc))

print('length X_test:',  len(X_test))
print('length y_test:', len(y_test))

length X_train: 1024
length y_train: 1024
length X_val: 256
length y_val: 256
length X_test: 549
length y_test: 549


In [ ]:
def classifier_summary(pipeline, X_train, y_train, X_val, y_val,X_test,y_test):

    start = time()
    sentiment_fit = pipeline.fit(X_train, y_train)
    end = time()

    y_pred_train= sentiment_fit.predict(X_train)
    y_pred_val = sentiment_fit.predict(X_val)
    y_pred_test = sentiment_fit.predict(X_test)
    
    train_accuracy = np.round(accuracy_score(y_train, y_pred_train),4)*100
    train_precision = np.round(precision_score(y_train, y_pred_train, average='weighted'),4)
    train_recall = np.round(recall_score(y_train, y_pred_train, average='weighted'),4)
    train_F1 = np.round(f1_score(y_train, y_pred_train, average='weighted'),4)
    train_kappa =  np.round(cohen_kappa_score(y_train, y_pred_train),4)
    train_confusion_matrix = confusion_matrix(y_train,y_pred_train)
    
    val_accuracy = np.round(accuracy_score(y_val, y_pred_val),4)*100
    val_precision = np.round(precision_score(y_val, y_pred_val, average='weighted'),4)
    val_recall = np.round(recall_score(y_val, y_pred_val, average='weighted'),4)
    val_F1 = np.round(f1_score(y_val, y_pred_val, average='weighted'),4)
    val_kappa =  np.round(cohen_kappa_score(y_val, y_pred_val),4)
    val_confusion_matrix = confusion_matrix(y_val,y_pred_val)
    
    test_accuracy = np.round(accuracy_score(y_test, y_pred_test),4)*100
    test_precision = np.round(precision_score(y_test, y_pred_test, average='weighted'),4)
    test_recall = np.round(recall_score(y_test, y_pred_test, average='weighted'),4)
    test_F1 = np.round(f1_score(y_test, y_pred_test, average='weighted'),4)
    test_kappa =  np.round(cohen_kappa_score(y_test, y_pred_test),4) 
    test_confusion_matrix = confusion_matrix(y_test,y_pred_test)
    
    
    print()
    print("Time taken : {}".format(end-start))
    print('------------------------ Train Set Metrics------------------------')
    print()
    print("accuracy : {}%".format(train_accuracy))
    print("F1_score : {}".format(train_F1))
    print("Cohen Kappa Score : {} ".format(train_kappa))
    print("Recall : {}".format(train_recall))
    print("Precision : {}".format(train_precision))
    print("Confusion Matrix :\n {}".format(train_confusion_matrix))
    
    print()
    print('------------------------ Validation Set Metrics------------------------')
    print()
    print("accuracy : {}%".format(val_accuracy))
    print("F1_score : {}".format(val_F1))
    print("Cohen Kappa Score : {} ".format(val_kappa))
    print("Recall : {}".format(val_recall))
    print("Precision : {}".format(val_precision))
    print("Confusion Matrix :\n {}".format(val_confusion_matrix))
    
    print()
    print('------------------------ Test Set Metrics------------------------')
    print()
    print("accuracy : {}%".format(test_accuracy))
    print("F1_score : {}".format(test_F1))
    print("Cohen Kappa Score : {} ".format(test_kappa))
    print("Recall : {}".format(test_recall))
    print("Precision : {}".format(test_precision))
    print("Confusion Matrix : {}".format(test_confusion_matrix))

    print("-"*80)
    print()

In [ ]:
names = [
        'SVM',
        "Random Forest Classifier",
        "AdaBoost Classifier", 
        "XGB Classifier",
         ]
classifiers = [
    SVC(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
        ]

zipped_clf = zip(names,classifiers)

In [ ]:
def classifier_comparator(X_train,y_train,X_val,y_val,X_test,y_test,classifier=zipped_clf): 
    result = []
    for n,c in classifier:
        checker_pipeline = Pipeline([
            ('classifier', c)
        ])
        print("Fitting {} on features ".format(n))
        #print(c)
        classifier_summary(checker_pipeline,X_train, y_train, X_val, y_val,X_test,y_test)
        
classifier_comparator(x_train_acc,y_train_acc,x_val_acc,y_val_acc,X_test,y_test,classifier=zipped_clf)

Fitting SVM on features 
